In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from pymongo import MongoClient
import pandas as pd
import numpy as np

import requests
import time
import json
from datetime import datetime, timedelta

In [ ]:
key = 'moby_dEWdtOk9rPY6zvkucUY971GFGuf'
offset = 0
endpoint = f"https://api.mobygames.com/v1/games?offset={offset}&format=brief&api_key={key}"

In [ ]:
def time_out():
    shift = start + timedelta(hours = 1) - datetime.now()
    time.sleep(shift.seconds)

In [ ]:
def make_request(endpoint, repite=False):
    response = requests.get(endpoint).content
    response = json.loads(response)
    if response.get('code') in [400]:
        print(
            f"Error {response.get('code')} : \n\t" + response.get("error") + "\n\tMessage:" + response.get("message") + "\n\t OFSET: " + offset
        )
        return {"status": "Error", "content": None}
    if response.get('code') in [401, 404, 422]:
        print(
            f"Error {response.get('code')} : \n\t" + response.get("error") + "\n\tMessage:" + response.get("message") + "\n\t OFSET: " + offset
        )
        raise Exception(
            f"Error {response.get('code')} : \n\t" + response.get("error") + "\n\tMessage:" + response.get("message") + "\n\t OFSET: " + offset
        )
    if response.get('code') == 429:
        print(
            f"Error {response.get('code')} : \n\t" + response.get("error") + "\n\tMessage:" + response.get("message") + "\n\t OFSET: " + offset
        )
        return {"status": "Error", "content": None}
    return {"status": "OK", "content": response.get("games")}

In [ ]:
games = []
for h in range(4):
    start = datetime.now()
    for i in range(360):
        endpoint = f"https://api.mobygames.com/v1/games?offset={offset}&format=brief&api_key={key}"
        r = make_request(endpoint)
        if r['status'] == "OK":
            games += r['content']
            offset += 100
        time.sleep(1)
    time_out()

In [ ]:
def prepare_tokens(v):
    v = v.lower()
    tokens = word_tokenize(v)
    return {"value": v, "tokens": tokens, "l": len(tokens)}

In [ ]:
games = [{**i,"type": 'game' ,**prepare_tokens(i["title"])} for i in games]

In [ ]:
with open("games.json", 'w') as f:
    json.dump(games, f)

In [4]:
client = MongoClient('localhost', 27017)

In [5]:
db = client['news_classification']['Tags']

In [18]:
with open("/home/vadim/Projects/news_classifire/Data/tags.json", 'r') as f:
    tags = json.load(f)
    tags = sorted(tags, key=lambda x: x['l'], reverse=True)
    counter = 0
    tags = [{"_id": tags.index(i)+1, **i} for i in tags if i['type'] == 'game']

In [19]:
db.insert_many(tags)